## imports

In [1]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np

## Parameters

In [12]:
# Instantiate the model
input_size = 400  # Example input size
hidden_size = 64  # Example hidden size
num_layers = 2    # Example number of layers
num_features = 2  # Number of features in each twin

## Twin Rnn Model

In [18]:
class TwinRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_features):
        super(TwinRNN, self).__init__()
        
        # Define LSTM layers for twin 1 and twin 2
        self.lstmTwin1 = nn.LSTM(num_features, hidden_size, num_layers, batch_first=True)
        self.lstmTwin2 = nn.LSTM(num_features, hidden_size, num_layers, batch_first=True)
        
        # Initialize hidden states and cell states
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.hidden1 = self.init_hidden()
        self.hidden2 = self.init_hidden()
        
    def init_hidden(self):
        # Initialize hidden state and cell state
        return (torch.zeros(self.num_layers, 1, self.hidden_size),
                torch.zeros(self.num_layers, 1, self.hidden_size))
        
    def forward(self, x):
        # Twin 1
        xTwin1 = x[:, :, :2]
        features1 = xTwin1[:, :, 0].unsqueeze(-1)  # Selecting the first feature column
        inputsTwin1 = features1.float()
        inputsTwin1 = inputsTwin1.permute(1, 0, 2)  # Transposing dimensions for LSTM input
        lstm_out1, (h_n1, c_n1) = self.lstmTwin1(inputsTwin1, self.hidden1)
        featMapTwin1 = torch.cat([h_n1[i, :, :] for i in range(h_n1.shape[0])], dim=1)
        
        # Twin 2
        xTwin2 = x[:, :, 2:4]
        features2 = xTwin2
        features2 = xTwin2[:, :, 0].unsqueeze(-1)  # Selecting the first feature column
        inputsTwin2 = inputsTwin2.permute(1, 0, 2)
        lstm_out2, (h_n2, c_n2) = self.lstmTwin2(inputsTwin2, self.hidden2)
        featMapTwin2 = torch.cat([h_n2[i, :, :] for i in range(h_n2.shape[0])], dim=1)
        
        # Combine twins
        featMap = torch.cat((featMapTwin1, featMapTwin2), dim=1)
        
        return featMap



        


## Data

In [10]:
# Load the CSV file
data = pd.read_csv('./data.csv')

data.head()

,num1,num2,label
0,81,67,0
1,67,25,1
2,93,78,0
3,58,54,0
4,23,83,1


## Train

In [19]:

# Convert the DataFrame to a PyTorch tensor
data_tensor = torch.tensor(data.values, dtype=torch.float32)


model = TwinRNN(input_size, hidden_size, num_layers, num_features)

# Perform inference
with torch.no_grad():
    output = model(data_tensor.unsqueeze(0))  # Add batch dimension
    print(output)

UnboundLocalError: cannot access local variable 'inputsTwin1' where it is not associated with a value